# Connect to google account

In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/data")

Mounted at /content/data


In [2]:
# 모듈 Import
import random
import numpy as np # 행렬 계산에 사용하는 모듈
import pandas as pd # 데이터 처리와 분석을 위한 모듈
import matplotlib.pyplot as plt # 데이터 시각화를 위한 모듈. 2D, 3D 그릴 때 사용
import seaborn as sns # 데이터 시각화를 위한 모듈. 두 데이터의 관계를 볼때 사용
from tqdm.auto import tqdm
from collections import defaultdict
from sklearn.decomposition import TruncatedSVD, NMF, SparsePCA
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 딕셔너리를 간편하게 사용하는 것을 도와준다. 모델 class 처럼 사용 가능
# dot을 이용해 객체를 불러 사용. JSON 다룰때 유용.
import easydict
args = easydict.EasyDict()

# path
args.default_path = "/content/data/MyDrive/Playdata/Competitions/ML/Dacon/" # 메인 경로
args.apply_train_path = args.default_path + "apply_train.csv" # train 데이터 경로
args.company_path = args.default_path + "company.csv"
args.recruitment_path = args.default_path + "recruitment.csv"
args.resume_certificate_path = args.default_path + "resume_certificate.csv"
args.resume_education_path = args.default_path + "resume_education.csv"
args.resume_language_path = args.default_path + "resume_language.csv"
args.resume_path = args.default_path + "resume.csv"

args.default_submission_path = args.default_path + "sample_submission.csv" # 예측결과(제출파일) 경로

# 데이터 분석을 위한 변수들
# # 난수 생성 제어 => 같은 코드를 실행해도 동일한 결과를 얻기 위해서 설정
# 데이터 분할 및 모델 초기화 때 유용.
args.random_state = 42
args.results = [] # 결과 저장 리스트

In [4]:
args.submission_path = args.default_path + "result/submission_Model_2.csv" # 결과 저장 파일
args.save_results = args.default_path+"result/model_results_Model_2.json" # 결과 저장 json

# File Load

In [5]:
apply_train_df = pd.read_csv(args.apply_train_path) # apply_train -> DataFrame화
company_df = pd.read_csv(args.company_path) # company -> DataFrame화
recruitment_df = pd.read_csv(args.recruitment_path) # recruitment -> DataFrame화
resume_certificate_df = pd.read_csv(args.resume_certificate_path) # resume_certificate -> DataFreame화
resume_education_df = pd.read_csv(args.resume_education_path) # resume_education -> DataFrame화
resume_language_df = pd.read_csv(args.resume_language_path) # resume_language -> DataFrame화
resume_df = pd.read_csv(args.resume_path) # resume -> DataFrame화


In [6]:
apply_train = apply_train_df.copy()
company = company_df.copy()
recruitment = recruitment_df.copy()
resume_certificate = resume_certificate_df.copy()
resume_education = resume_education_df.copy()
resume_language = resume_language_df.copy()
resume = resume_df.copy()

# 탐색

### Negative Sampling

In [7]:
apply_train.head()

,resume_seq,recruitment_seq
0,U05833,R03838
1,U06456,R02144
2,U07807,R01877
3,U04842,R02463
4,U08336,R00112


In [8]:
resume_pool = set(apply_train["resume_seq"].unique())
recruitment_pool = set(apply_train['recruitment_seq'].unique())

len(resume_pool), len(recruitment_pool)

(8482, 6695)

#### apply_train에서 resume_seq 기준으로 지원한 공고, 지원하지 않은 공고 추출

In [9]:
df_add_negative = apply_train.groupby(["resume_seq"])["recruitment_seq"].apply(set).reset_index().rename(columns={"recruitment_seq":"interacted_iid"})
df_add_negative

,resume_seq,interacted_iid
0,U00001,"{R04536, R05288, R06065, R05210}"
1,U00002,"{R04588, R05472, R06019, R01960, R02346, R0294..."
2,U00003,"{R04918, R01460, R03301}"
3,U00004,"{R00312, R00004, R01825, R05341, R06658, R0200..."
4,U00005,"{R03914, R00832, R00374}"
...,...,...
8477,U08478,"{R02170, R03939}"
8478,U08479,"{R04196, R04036, R03837, R02787, R02988, R01470}"
8479,U08480,"{R00803, R03152}"
8480,U08481,"{R05500, R00225, R06253}"


In [10]:
# 지원하지 않은 공고
df_add_negative['negative_iid'] = df_add_negative['interacted_iid'].map(lambda x: recruitment_pool - x)
df_add_negative

,resume_seq,interacted_iid,negative_iid
0,U00001,"{R04536, R05288, R06065, R05210}","{R04702, R02063, R03707, R04734, R02113, R0079..."
1,U00002,"{R04588, R05472, R06019, R01960, R02346, R0294...","{R04702, R02063, R03707, R04734, R02113, R0079..."
2,U00003,"{R04918, R01460, R03301}","{R04702, R02063, R03707, R04734, R02113, R0079..."
3,U00004,"{R00312, R00004, R01825, R05341, R06658, R0200...","{R04702, R02063, R03707, R04734, R02113, R0079..."
4,U00005,"{R03914, R00832, R00374}","{R04702, R02063, R03707, R04734, R02113, R0079..."
...,...,...,...
8477,U08478,"{R02170, R03939}","{R04702, R02063, R03707, R04734, R02113, R0079..."
8478,U08479,"{R04196, R04036, R03837, R02787, R02988, R01470}","{R04702, R02063, R03707, R04734, R02113, R0079..."
8479,U08480,"{R00803, R03152}","{R04702, R02063, R03707, R04734, R02113, R0079..."
8480,U08481,"{R05500, R00225, R06253}","{R04702, R02063, R03707, R04734, R02113, R0079..."


In [11]:
# 지원한 공고의 수
df_add_negative['interacted_iid_cnt'] = df_add_negative['interacted_iid'].map(lambda x: len(x))
df_add_negative

,resume_seq,interacted_iid,negative_iid,interacted_iid_cnt
0,U00001,"{R04536, R05288, R06065, R05210}","{R04702, R02063, R03707, R04734, R02113, R0079...",4
1,U00002,"{R04588, R05472, R06019, R01960, R02346, R0294...","{R04702, R02063, R03707, R04734, R02113, R0079...",8
2,U00003,"{R04918, R01460, R03301}","{R04702, R02063, R03707, R04734, R02113, R0079...",3
3,U00004,"{R00312, R00004, R01825, R05341, R06658, R0200...","{R04702, R02063, R03707, R04734, R02113, R0079...",17
4,U00005,"{R03914, R00832, R00374}","{R04702, R02063, R03707, R04734, R02113, R0079...",3
...,...,...,...,...
8477,U08478,"{R02170, R03939}","{R04702, R02063, R03707, R04734, R02113, R0079...",2
8478,U08479,"{R04196, R04036, R03837, R02787, R02988, R01470}","{R04702, R02063, R03707, R04734, R02113, R0079...",6
8479,U08480,"{R00803, R03152}","{R04702, R02063, R03707, R04734, R02113, R0079...",2
8480,U08481,"{R05500, R00225, R06253}","{R04702, R02063, R03707, R04734, R02113, R0079...",3


In [12]:
# 지원한 공고 횟수만큼 지원하지 않은 회사 추출..
df_add_negative['negative_sampling'] = df_add_negative.apply(lambda row: random.sample(list(row['negative_iid']), row['interacted_iid_cnt']), axis=1)

In [13]:
df_add_negative.head()

,resume_seq,interacted_iid,negative_iid,interacted_iid_cnt,negative_sampling
0,U00001,"{R04536, R05288, R06065, R05210}","{R04702, R02063, R03707, R04734, R02113, R0079...",4,"[R02158, R04499, R04492, R04034]"
1,U00002,"{R04588, R05472, R06019, R01960, R02346, R0294...","{R04702, R02063, R03707, R04734, R02113, R0079...",8,"[R03212, R06087, R00439, R03035, R02055, R0422..."
2,U00003,"{R04918, R01460, R03301}","{R04702, R02063, R03707, R04734, R02113, R0079...",3,"[R02012, R01565, R05822]"
3,U00004,"{R00312, R00004, R01825, R05341, R06658, R0200...","{R04702, R02063, R03707, R04734, R02113, R0079...",17,"[R03971, R04369, R06665, R03393, R04113, R0445..."
4,U00005,"{R03914, R00832, R00374}","{R04702, R02063, R03707, R04734, R02113, R0079...",3,"[R06246, R06033, R02216]"


In [14]:
df_add_negative[['resume_seq','interacted_iid', 'negative_sampling']].head()

,resume_seq,interacted_iid,negative_sampling
0,U00001,"{R04536, R05288, R06065, R05210}","[R02158, R04499, R04492, R04034]"
1,U00002,"{R04588, R05472, R06019, R01960, R02346, R0294...","[R03212, R06087, R00439, R03035, R02055, R0422..."
2,U00003,"{R04918, R01460, R03301}","[R02012, R01565, R05822]"
3,U00004,"{R00312, R00004, R01825, R05341, R06658, R0200...","[R03971, R04369, R06665, R03393, R04113, R0445..."
4,U00005,"{R03914, R00832, R00374}","[R06246, R06033, R02216]"


In [15]:
# 리스트인 interacted_iid 컬럼을 각각 row로 변경한다.
df_interacted = df_add_negative[['resume_seq', 'interacted_iid']].explode('interacted_iid').rename(
        columns={'interacted_iid':'recruitment_seq'}
    )
# 지원한 회사이므로 target 컬럼의 모든 값은 1
df_interacted['target'] = 1
# 인덱스 재정렬
df_interacted.reset_index(drop=True, inplace=True)

print(df_interacted.shape)
df_interacted.head()

(57946, 3)


,resume_seq,recruitment_seq,target
0,U00001,R04536,1
1,U00001,R05288,1
2,U00001,R06065,1
3,U00001,R05210,1
4,U00002,R04588,1


In [16]:
# 리스트인 negative_sampling 컬럼을 각각 row로 변경한다.
df_negatived = df_add_negative[['resume_seq', 'negative_sampling']].explode('negative_sampling').rename(
        columns={'negative_sampling':'recruitment_seq'}
    )
# 지원하지 않은 공고이므로 target은 0
df_negatived['target'] = 0
# 인덱스 재정렬
df_negatived.reset_index(drop=True, inplace=True)

print(df_negatived.shape)
df_negatived.head()

(57946, 3)


,resume_seq,recruitment_seq,target
0,U00001,R02158,0
1,U00001,R04499,0
2,U00001,R04492,0
3,U00001,R04034,0
4,U00002,R03212,0


In [17]:
# 지원한 공고 df, 지원하지 않은 공고 df를 합친다.
# axis = 0 : row로 합친다.
df_concat = pd.concat([df_interacted, df_negatived], axis=0)

print(df_concat.shape)
df_concat.head()

(115892, 3)


,resume_seq,recruitment_seq,target
0,U00001,R04536,1
1,U00001,R05288,1
2,U00001,R06065,1
3,U00001,R05210,1
4,U00002,R04588,1


In [18]:
# df_concat.sample(frac=1) : df전체를 무작위 샘플로 생성
# 인덱스 재정렬
df_shuffle = df_concat.sample(frac=1).reset_index(drop=True)

print(df_shuffle.shape)
df_shuffle.head()

(115892, 3)


,resume_seq,recruitment_seq,target
0,U08003,R00546,0
1,U03146,R00910,1
2,U01249,R06080,1
3,U05670,R00762,1
4,U02490,R06618,0


In [19]:
df_shuffle['target'].value_counts()

0    57946
1    57946
Name: target, dtype: int64

### recruitment_seq 기준 합치기

In [20]:
# recruitment, company 정렬(recruitment_seq 기준)
company = company.sort_values(by = "recruitment_seq")
recruitment = recruitment.sort_values(by = "recruitment_seq")

In [21]:
# merged_recruitment = recruitment, company merge
merged_recruitment = pd.merge(recruitment, company, on='recruitment_seq', how='left')
merged_recruitment.set_index('recruitment_seq', inplace=True)
merged_recruitment

,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword,company_type_seq,supply_kind,employee
recruitment_seq,,,,,,,,,,,,,
R00001,5.0,NaN,NaN,0,0,2101;2108;2201;2204;2205;2707;2810,2,2,1,NaN,5.0,201.0,631.0
R00002,3.0,NaN,NaN,0,0,2507;2703;2707,3,2,1,NaN,2.0,201.0,160.0
R00003,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,2,NaN,NaN,NaN,NaN
R00004,3.0,NaN,NaN,0,0,2507;2707,3,2,1,NaN,2.0,402.0,500.0
R00005,3.0,NaN,NaN,0,0,2507;2707,3,2,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R06691,3.0,NaN,NaN,0,0,2501;2507;2707,3,2,1,NaN,NaN,NaN,NaN
R06692,3.0,NaN,NaN,0,0,2201;2507,3,2,2,NaN,4.0,402.0,150.0
R06693,5.0,NaN,NaN,0,0,2102;2707,4,2,1,NaN,NaN,NaN,NaN


### resume_seq 기준 합치기

In [22]:
# resume 정렬 (resume_seq 기준)
resume = resume.sort_values(by='resume_seq')

In [23]:
# resume_certificate 정렬 (resume_seq 기준)
resume_certificate = resume_certificate.sort_values(by='resume_seq')
# certificate_contents에 결측값이 존재하는 행 제거
resume_certificate = resume_certificate.dropna(subset=['certificate_contents'])
# 이력서번호(resume_seq)기준으로 groupby
# 이력서번호(resume_seq)의 해당하는 자경증들을 세미콜론(;)으로 join해서 적용(apply)
resume_certificate = resume_certificate.groupby('resume_seq')['certificate_contents'].apply(';'.join).reset_index()

In [24]:
# resume_education 정렬 (resume_seq 기준)
resume_education = resume_education.sort_values(by='resume_seq')

In [25]:
# resume_language 정렬 (resume_seq 기준)
resume_language = resume_language.sort_values(by='resume_seq')
# resume_seq, lanauge;exam_name;score
# lanauge;exam_name;score값을 넣을 컬럼 생성
resume_language['lang_exam_score'] = resume_language['language'].astype(str) + ';' + resume_language['exam_name'].astype(str) + ';' + resume_language['score'].astype(str)
# 기존 lanauge, exam_name, score 컬럼 삭제
resume_language = resume_language.drop(['language','exam_name', 'score', 'score'], axis=1)
# lang_exam_score 에 결측값이 존재하는 행 제거
resume_language = resume_language.dropna(subset=['lang_exam_score'])
# 한 유저가 여러개 자격증정보를 가지고 있으니 &로 묶기
resume_language = resume_language.groupby('resume_seq')['lang_exam_score'].apply('&'.join).reset_index()

In [26]:
# merged_resume = resume, resume_certificate, resume_education, resume_language
merged_resume = pd.merge(resume, resume_certificate, on='resume_seq', how='left')
merged_resume = pd.merge(merged_resume, resume_education, on='resume_seq', how='left')
merged_resume = pd.merge(merged_resume, resume_language, on='resume_seq', how='left')
merged_resume.set_index('resume_seq', inplace=True)
merged_resume

,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,...,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score,lang_exam_score
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,2019-12-27,2020-02-01,4,2009,1500.0,1500.0,디자이너,재료·화학·섬유·의복,NaN,NaN,...,4,5,5,0,17,NaN,NaN,9,20.0,NaN
U00002,2020-04-24,2020-04-29,4,2005,0.0,0.0,디자이너,재료·화학·섬유·의복,NaN,NaN,...,0,5,5,0,3,NaN,NaN,8,90.0,NaN
U00003,2018-02-14,2020-07-08,4,2004,0.0,0.0,남성복디자이너;TD캐주얼,재료·화학·섬유·의복,NaN,NaN,...,6,5,5,0,17,NaN,NaN,4,90.0,NaN
U00004,2017-10-26,2020-04-27,4,2011,0.0,0.0,상품기획;영업기획,재료·화학·섬유·의복,NaN,NaN,...,5,5,5,0,5,NaN,NaN,3,70.0,2;11;661.62
U00005,2020-03-18,2020-04-08,4,2011,0.0,0.0,인사;총무;경영;MD;상품기획,재료·화학·섬유·의복,NaN,NaN,...,5,0,0,0,0,NaN,NaN,10,50.0,2;4;873.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,2020-02-05,2020-03-31,4,2003,0.0,2700.0,상품기획,재료·화학·섬유·의복,NaN,NaN,...,3,5,5,0,5,NaN,NaN,19,0.0,NaN
U08479,2016-02-17,2020-07-15,3,0,2500.0,2300.0,디자이너,재료·화학·섬유·의복,NaN,NaN,...,3,3,6,0,3,가정과,NaN,9,0.0,NaN
U08480,2019-11-13,2020-06-24,4,0,0.0,0.0,VMD;텍스타일디자인;섬유디자인;니트디자인;여성복디자인,디자인,NaN,NaN,...,0,5,5,0,10,미술대학섬유패션코디네이션학과,NaN,9,80.0,NaN


### merged_recruitment 내용 확인

In [27]:
merged_recruitment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6695 entries, R00001 to R06695
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   address_seq1       6694 non-null   float64
 1   address_seq2       100 non-null    float64
 2   address_seq3       9 non-null      float64
 3   career_end         6695 non-null   int64  
 4   career_start       6695 non-null   int64  
 5   check_box_keyword  6695 non-null   object 
 6   education          6695 non-null   int64  
 7   major_task         6695 non-null   int64  
 8   qualifications     6695 non-null   int64  
 9   text_keyword       707 non-null    object 
 10  company_type_seq   2377 non-null   float64
 11  supply_kind        2377 non-null   float64
 12  employee           2377 non-null   float64
dtypes: float64(6), int64(5), object(2)
memory usage: 732.3+ KB


In [28]:
merged_recruitment.describe()

,address_seq1,address_seq2,address_seq3,career_end,career_start,education,major_task,qualifications,company_type_seq,supply_kind,employee
count,6694.000000,100.000000,9.000000,6695.0,6695.0,6695.000000,6695.000000,6695.000000,2377.000000,2377.000000,2377.000000
mean,3.103675,7.900000,16.555556,0.0,0.0,3.081703,2.277521,1.317401,2.944468,344.090029,217.696676
std,0.955459,7.399836,6.424778,0.0,0.0,0.753074,1.242267,0.477852,1.739406,131.232212,331.153073
min,1.000000,1.000000,3.000000,0.0,0.0,2.000000,1.000000,0.000000,2.000000,100.000000,0.000000
25%,3.000000,1.000000,18.000000,0.0,0.0,3.000000,2.000000,1.000000,2.000000,402.000000,41.000000
50%,3.000000,5.000000,20.000000,0.0,0.0,3.000000,2.000000,1.000000,2.000000,402.000000,145.000000
75%,3.000000,14.000000,20.000000,0.0,0.0,4.000000,2.000000,2.000000,4.000000,402.000000,300.000000
max,20.000000,20.000000,20.000000,0.0,0.0,6.000000,10.000000,2.000000,9.000000,605.000000,12223.000000


In [29]:
(merged_recruitment.isnull().sum() / len(merged_recruitment)).sort_values()

career_end           0.000000
career_start         0.000000
check_box_keyword    0.000000
education            0.000000
major_task           0.000000
qualifications       0.000000
address_seq1         0.000149
company_type_seq     0.644959
supply_kind          0.644959
employee             0.644959
text_keyword         0.894399
address_seq2         0.985063
address_seq3         0.998656
dtype: float64

- 결측치가 90프로 이상인 컬럼 2개 존재
  - address_seq2, address_seq3 : drop 해도 괜찮을 것 같다.
- text_keyword(모집직무키워드) : 키워드 정리할 수 있는 방법
- employee(종업원수), supply_kind(주업종코드), company_type_seq(회사유형코드) : 동일한 index의 값이 결측치다. 분포도를 확인하여 채울 것인지 drop 할건지 결정
- address_seq1(근무지주소) : mode로 해도 될듯하다.

[other col]
- career_start, career_end : 값이 다 없는 상황이기에 drop 해도 괜찮을 것 같다.
- education : 요구학위와 resume의 최종학력과의 관계?
- company_type_seq : 중소, 중견, 대기업 구분?

### merged_resume 내용 확인

In [30]:
merged_resume.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8482 entries, U00001 to U08482
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   reg_date               8482 non-null   object 
 1   updated_date           8482 non-null   object 
 2   degree                 8482 non-null   int64  
 3   graduate_date          8482 non-null   int64  
 4   hope_salary            8482 non-null   float64
 5   last_salary            8482 non-null   float64
 6   text_keyword           8287 non-null   object 
 7   job_code_seq1          8482 non-null   object 
 8   job_code_seq2          500 non-null    object 
 9   job_code_seq3          187 non-null    object 
 10  career_month           8482 non-null   int64  
 11  career_job_code        7724 non-null   object 
 12  certificate_contents   5976 non-null   object 
 13  hischool_type_seq      8482 non-null   int64  
 14  hischool_special_type  8482 non-null   object 
 15  hi

In [31]:
merged_resume.describe()

,degree,graduate_date,hope_salary,last_salary,career_month,hischool_type_seq,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major_type,univ_score
count,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000,8482.000000
mean,3.938222,1819.586065,1395.095496,2505.706201,69.408984,16.402499,5.738387,4.204551,4.684626,0.004480,6.651733,8.095143,71.380350
std,0.440764,584.009224,1865.951253,2010.082733,63.547948,8.485975,4.886454,1.608187,1.892880,0.066787,5.272027,5.152574,20.418688
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2001.000000,0.000000,0.000000,18.000000,20.000000,3.000000,5.000000,5.000000,0.000000,3.000000,4.000000,70.000000
50%,4.000000,2007.000000,0.000000,2700.000000,53.000000,21.000000,4.000000,5.000000,5.000000,0.000000,5.000000,9.000000,80.000000
75%,4.000000,2011.000000,2700.000000,3700.000000,108.000000,21.000000,9.000000,5.000000,5.000000,0.000000,10.000000,9.000000,80.000000
max,6.000000,2022.000000,10000.000000,10000.000000,432.000000,23.000000,18.000000,6.000000,12.000000,1.000000,19.000000,23.000000,100.000000


In [32]:
merged_resume.columns

Index(['reg_date', 'updated_date', 'degree', 'graduate_date', 'hope_salary',
       'last_salary', 'text_keyword', 'job_code_seq1', 'job_code_seq2',
       'job_code_seq3', 'career_month', 'career_job_code',
       'certificate_contents', 'hischool_type_seq', 'hischool_special_type',
       'hischool_nation', 'hischool_gender', 'hischool_location_seq',
       'univ_type_seq1', 'univ_type_seq2', 'univ_transfer', 'univ_location',
       'univ_major', 'univ_sub_major', 'univ_major_type', 'univ_score',
       'lang_exam_score'],
      dtype='object')

In [33]:
(merged_resume.isnull().sum() / len(merged_resume)).sort_values()

reg_date                 0.000000
univ_major_type          0.000000
univ_location            0.000000
univ_transfer            0.000000
univ_type_seq2           0.000000
univ_type_seq1           0.000000
hischool_location_seq    0.000000
hischool_gender          0.000000
hischool_nation          0.000000
hischool_special_type    0.000000
univ_score               0.000000
hischool_type_seq        0.000000
career_month             0.000000
job_code_seq1            0.000000
last_salary              0.000000
hope_salary              0.000000
graduate_date            0.000000
degree                   0.000000
updated_date             0.000000
text_keyword             0.022990
career_job_code          0.089366
certificate_contents     0.295449
univ_major               0.788375
lang_exam_score          0.903325
job_code_seq2            0.941052
univ_sub_major           0.950955
job_code_seq3            0.977953
dtype: float64

- 결측치가 90프로 이상인 컬럼이 4개 존재
  - job_code_seq2, job_code_seq3(희망직무) : drop 해도 괜찮을 것 같다.
  - univ_sub_major(부전공) : drop 해도 괜찮을 것 같다.
  - lang_exam_score(언어,시험종류,점수) : 새로 만든 feature, 언어자격증 유무로 해야할지, drop 해야할지?

- univ_major(전공) : univ_type_seq 때문에 결측치가 많아 보인다. => drop 해도 괜찮을 것 같다.
- certificate_contents(자격증) : 자격증 유무?, 자격증 갯수, 키워드 정리할 수 있는 방법
- career_job_code(경력직무) : 키워드 정리할 수 있는 방법
- text_keyword(직무키워드) : 키워드 정리할 수 있는 방법

[other col]

- updated_date, reg_date : 굳이 학습할 필요가 있을까?
- hischool_location_seq : 고등학교 지역코드 기준으로 달라질까?
- univ_type_seq1 : 대학종류로 기준

### total_merge = df_shuffle, merged_resume, merged_recruitment merge

In [39]:
total_merge = df_shuffle.copy()
re_merged = merged_resume.copy()
recru_merged = merged_recruitment.copy()

In [40]:
re_merged.shape

(8482, 27)

In [41]:
recru_merged.shape

(6695, 13)

In [43]:
total_merge

,resume_seq,recruitment_seq,target
0,U08003,R00546,0
1,U03146,R00910,1
2,U01249,R06080,1
3,U05670,R00762,1
4,U02490,R06618,0
...,...,...,...
115887,U04241,R05815,0
115888,U04446,R06583,0
115889,U05772,R06684,0
115890,U05960,R05439,1


#### total_merge에 merged_resume 데이터 합치기

In [44]:
total_merge = pd.merge(total_merge, re_merged, on = "resume_seq", how = "left")
total_merge.reset_index(drop=True, inplace=True)
total_merge

,resume_seq,recruitment_seq,target,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,...,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score,lang_exam_score
0,U08003,R00546,0,2014-03-03,2020-04-17,4,2009,6250.0,5250.0,상품기획;기획MD,...,9,3,5,0,11,윤리교육과,NaN,17,80.0,NaN
1,U03146,R00910,1,2019-11-16,2020-06-18,4,2016,0.0,0.0,상품기획,...,3,5,5,0,18,NaN,NaN,9,70.0,NaN
2,U01249,R06080,1,2020-05-11,2020-08-20,3,2009,0.0,1700.0,패션디자이너,...,13,4,5,0,13,연기예술,NaN,9,80.0,NaN
3,U05670,R00762,1,2018-05-21,2020-01-08,4,2012,0.0,3300.0,디자이너;디자인;캐쥬얼;여성복;유니스타일,...,4,3,5,0,4,NaN,NaN,5,80.0,NaN
4,U02490,R06618,0,2017-06-12,2020-04-11,4,2009,0.0,2500.0,영업및영업MD,...,10,5,5,0,10,NaN,NaN,5,80.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115887,U04241,R05815,0,2014-09-02,2020-08-31,4,2004,0.0,3100.0,VMD,...,3,5,5,0,5,NaN,NaN,9,80.0,NaN
115888,U04446,R06583,0,2017-10-16,2020-09-28,3,2003,5250.0,4500.0,매장영업,...,5,0,0,0,0,NaN,NaN,20,80.0,NaN
115889,U05772,R06684,0,2019-11-12,2020-08-10,4,2015,0.0,3100.0,여성복;기획MD;상품기획;머천다이저,...,5,5,5,0,3,NaN,NaN,3,70.0,NaN
115890,U05960,R05439,1,2019-05-26,2020-03-22,5,2004,0.0,3700.0,바이어;상품기획;머천다이저,...,0,5,5,0,12,NaN,NaN,4,90.0,NaN


#### total_merge에 merged_recruitment 데이터 합치기

In [46]:
total_merge = pd.merge(total_merge, recru_merged, on = "recruitment_seq", how = "left")
total_merge.reset_index(drop=True, inplace=True)
total_merge

,resume_seq,recruitment_seq,target,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword_x,...,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword_y,company_type_seq,supply_kind,employee
0,U08003,R00546,0,2014-03-03,2020-04-17,4,2009,6250.0,5250.0,상품기획;기획MD,...,0,0,2201;2204;2205;2707,3,2,1,NaN,NaN,NaN,NaN
1,U03146,R00910,1,2019-11-16,2020-06-18,4,2016,0.0,0.0,상품기획,...,0,0,2101;2108;2201;2501;2507;2707,4,2,1,NaN,NaN,NaN,NaN
2,U01249,R06080,1,2020-05-11,2020-08-20,3,2009,0.0,1700.0,패션디자이너,...,0,0,2507;2707,4,8,1,NaN,2.0,402.0,130.0
3,U05670,R00762,1,2018-05-21,2020-01-08,4,2012,0.0,3300.0,디자이너;디자인;캐쥬얼;여성복;유니스타일,...,0,0,2507;2707,3,2,1,NaN,NaN,NaN,NaN
4,U02490,R06618,0,2017-06-12,2020-04-11,4,2009,0.0,2500.0,영업및영업MD,...,0,0,2101;2108;2201;2707,4,2,2,MD,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115887,U04241,R05815,0,2014-09-02,2020-08-31,4,2004,0.0,3100.0,VMD,...,0,0,2101;2108;2201;2707,3,2,2,NaN,NaN,NaN,NaN
115888,U04446,R06583,0,2017-10-16,2020-09-28,3,2003,5250.0,4500.0,매장영업,...,0,0,2108;2707;2810,3,2,1,NaN,NaN,NaN,NaN
115889,U05772,R06684,0,2019-11-12,2020-08-10,4,2015,0.0,3100.0,여성복;기획MD;상품기획;머천다이저,...,0,0,2201;2204;2205;2707,4,2,1,NaN,NaN,NaN,NaN
115890,U05960,R05439,1,2019-05-26,2020-03-22,5,2004,0.0,3700.0,바이어;상품기획;머천다이저,...,0,0,2104;2203,2,2,2,NaN,NaN,NaN,NaN


## EDA (train data)

## 결측치 제거

### merged_recruitment

In [ ]:
# 컬럼제거
drop_col = ["address_seq2", "address_seq3", "text_keyword", "career_end", "career_start"]
merged_recruitment.drop(columns = drop_col, inplace = True)

In [ ]:
addr_cnt = merged_recruitment["address_seq1"].value_counts()
addr_cnt

3.0     6490
5.0      174
20.0      17
11.0       5
1.0        4
13.0       2
2.0        1
9.0        1
Name: address_seq1, dtype: int64

In [ ]:
com_type_cnt = merged_recruitment["company_type_seq"].value_counts()
com_type_cnt

2.0    1658
4.0     410
5.0     161
9.0     109
7.0      35
3.0       4
Name: company_type_seq, dtype: int64

In [ ]:
supply_kind_cnt = merged_recruitment["supply_kind"].value_counts()
supply_kind_cnt

402.0    1606
100.0     463
514.0     126
201.0     101
508.0      33
501.0      20
515.0       6
601.0       5
409.0       5
507.0       4
413.0       2
502.0       1
513.0       1
605.0       1
603.0       1
512.0       1
412.0       1
Name: supply_kind, dtype: int64

In [ ]:
employee_cnt = merged_recruitment["employee"].value_counts(normalize=True)
employee_cnt

90.0     0.080774
20.0     0.074884
250.0    0.068153
150.0    0.062684
590.0    0.059739
           ...   
280.0    0.000421
23.0     0.000421
18.0     0.000421
45.0     0.000421
11.0     0.000421
Name: employee, Length: 81, dtype: float64

In [ ]:
addr_mode = merged_recruitment["address_seq1"].mode().values[0]
company_type_mode = merged_recruitment["company_type_seq"].mode().values[0]
supply_kind_mode = merged_recruitment["supply_kind"].mode().values[0]
employee_median = merged_recruitment["employee"].median()

In [ ]:
merged_recruitment["address_seq1"].fillna(addr_mode, inplace = True)
merged_recruitment["company_type_seq"].fillna(company_type_mode, inplace = True)
merged_recruitment["supply_kind"].fillna(supply_kind_mode, inplace = True)
merged_recruitment["employee"].fillna(employee_median, inplace = True)

In [ ]:
(merged_recruitment.isnull().sum() / len(merged_recruitment)).sort_values()

address_seq1         0.0
check_box_keyword    0.0
education            0.0
major_task           0.0
qualifications       0.0
company_type_seq     0.0
supply_kind          0.0
employee             0.0
dtype: float64

### merged_resume

In [ ]:
drop_col = ["job_code_seq3", "univ_sub_major", "job_code_seq2", "lang_exam_score", "univ_major" ]
merged_resume.drop(columns = drop_col, inplace = True)

In [ ]:
(merged_resume.isnull().sum() / len(merged_resume)).sort_values()

reg_date                 0.000000
univ_location            0.000000
univ_transfer            0.000000
univ_type_seq2           0.000000
univ_type_seq1           0.000000
hischool_location_seq    0.000000
hischool_gender          0.000000
hischool_nation          0.000000
hischool_special_type    0.000000
univ_major_type          0.000000
hischool_type_seq        0.000000
career_month             0.000000
job_code_seq1            0.000000
last_salary              0.000000
hope_salary              0.000000
graduate_date            0.000000
degree                   0.000000
updated_date             0.000000
univ_score               0.000000
text_keyword             0.022990
career_job_code          0.089366
certificate_contents     0.295449
univ_major               0.788375
dtype: float64

## 신규컬럼 생성

# 앙상블 Modeling (train_f, train_t)
- shape, 결측치 확인 필수

## 데이터 분리 (train data)

## Experiment

- 필요한 라이브러리 import

### Cross Validation

### Model

### HPO

## Best Model 해석

### Cross Validation

### Best Model 생성 및 학습

### Confusion Matrix

### HeatMap by Confusion Matrix

# 예측

# 평가

In [ ]:
def recall5(answer_df, submission_df):
    """
    Calculate recall@5 for given dataframes.

    Parameters:
    - answer_df: DataFrame containing the ground truth
    - submission_df: DataFrame containing the predictions

    Returns:
    - recall: Recall@5 value
    """

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # submission의 예측이 각각 5개인지 확인
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # submission의 예측된 값들에 null값이 있는지 확인
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # 예측값에 중복이 있는지 확인
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # primary_col 즉 resume_seq가 양측에 있는지 확인 후 남김
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

# Result 저장 (Submission)
- Dictionary List로 sort해서 best model select